In [ ]:
from ultralytics import YOLO 
import os
import pandas as pd
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [2]:
model = YOLO("yolo12n")

In [3]:
model_train = model.train(data="data.yaml",epochs=100,imgsz=500,batch=50,patience=20)

New https://pypi.org/project/ultralytics/8.3.199 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.189  Python-3.10.10 torch-2.8.0+cpu CPU (13th Gen Intel Core(TM) i7-1355U)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=50, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=500, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer

In [4]:
model = YOLO("D:\\My projects\\Skin cancer\\runs\\detect\\train\\weights\\best.pt")
result = model.predict("D:\\My projects\\Skin cancer\\Test\\images",save=True)


image 1/318 D:\My projects\Skin cancer\Test\images\Herpes (100).jpg: 352x512 3 Herpess, 165.9ms
image 2/318 D:\My projects\Skin cancer\Test\images\Herpes (101).jpg: 352x512 1 Herpes, 132.3ms
image 3/318 D:\My projects\Skin cancer\Test\images\Herpes (102).jpg: 384x512 4 Herpess, 225.9ms
image 4/318 D:\My projects\Skin cancer\Test\images\Herpes (107).jpg: 352x512 4 Herpess, 109.0ms
image 5/318 D:\My projects\Skin cancer\Test\images\Herpes (108).jpg: 288x512 6 Herpess, 128.4ms
image 6/318 D:\My projects\Skin cancer\Test\images\Herpes (109).jpg: 416x512 4 Herpess, 165.6ms
image 7/318 D:\My projects\Skin cancer\Test\images\Herpes (11).png: 320x512 9 Herpess, 128.9ms
image 8/318 D:\My projects\Skin cancer\Test\images\Herpes (110).jpg: 448x512 2 Herpess, 155.9ms
image 9/318 D:\My projects\Skin cancer\Test\images\Herpes (111).jpg: 288x512 4 Herpess, 80.4ms
image 10/318 D:\My projects\Skin cancer\Test\images\Herpes (112).jpg: 320x512 3 Herpess, 84.6ms
image 11/318 D:\My projects\Skin cancer\Te

In [14]:
data = []

for i in result:
    path_file = os.path.basename(i.path)
    for box,conf in zip(i.boxes.xyxy,i.boxes.conf):
        x1,y1,x2,y2 = map(int,box)
        confidence = float(conf)
        data.append([path_file,confidence,x1,y1,x2,y2])

In [15]:
df = pd.DataFrame(data,columns=["Image","Confidence","X1","Y1","X2","Y2"])

In [16]:
df

,Image,Confidence,X1,Y1,X2,Y2
0,Herpes (100).jpg,0.595933,138,111,198,165
1,Herpes (100).jpg,0.403773,91,46,145,115
2,Herpes (100).jpg,0.392276,154,19,270,113
3,Herpes (101).jpg,0.795939,58,36,120,122
4,Herpes (102).jpg,0.630074,43,127,91,184
...,...,...,...,...,...,...
1591,Varicela (8).jpg,0.310230,566,600,618,647
1592,Varicela (8).jpg,0.286839,55,483,114,538
1593,Varicela (8).jpg,0.278614,387,562,466,636
1594,Varicela (9).png,0.334816,115,86,129,105


In [ ]:
df.to_csv("data_skin.csv",index=False)

In [41]:
output_dir = "cropped_dataset"
os.makedirs(output_dir, exist_ok=True)

for i in result:
    img = Image.open(i.path)
    path_file = os.path.basename(i.path)
    for idx, box in enumerate(i.boxes.xyxy):
        x1, y1, x2, y2 = map(int, box)
        cropped = img.crop((x1, y1, x2, y2)).convert("RGB")
        cropped.save(os.path.join(output_dir, f"{path_file}_{idx}.png"))

In [70]:
img_size = 128
batch_size = 32


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "cropped_dataset",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    "cropped_dataset",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 1278 images belonging to 4 classes.
Found 318 images belonging to 4 classes.


In [66]:
true_classes = val_generator.classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(true_classes),
    y=true_classes
)
class_weights_dict = dict(enumerate(class_weights))

In [74]:
weights_path = "mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5"

base_model = MobileNetV2(
    weights=None,
    include_top=False,
    input_shape=(img_size, img_size, 3)
)
base_model.load_weights(weights_path)

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False

In [75]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [76]:
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    class_weight=class_weights_dict
)

C:\Users\AliReza\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 18s 358ms/step - accuracy: 0.5477 - loss: 0.9343 - val_accuracy: 0.5629 - val_loss: 0.9376
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 307ms/step - accuracy: 0.7011 - loss: 0.5745 - val_accuracy: 0.5692 - val_loss: 0.9968
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 309ms/step - accuracy: 0.7285 - loss: 0.5015 - val_accuracy: 0.5535 - val_loss: 1.0030
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 311ms/step - accuracy: 0.7308 - loss: 0.4921 - val_accuracy: 0.6321 - val_loss: 0.9153
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 309ms/step - accuracy: 0.7488 - loss: 0.5128 - val_accuracy: 0.6887 - val_loss: 0.8715
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 328ms/step - accuracy: 0.7801 - loss: 0.4108 - val_accuracy: 0.6509 - val_loss: 0.9114
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 13s 334ms/step - accuracy: 0.8075 - loss: 0.3878 - val_accuracy: 0.6447 - val_loss: 0.9071
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 345ms/step - accuracy: 0.8052 - loss: 0.3754 - val_accu

In [77]:
model.save("skin_lesion_classifier.keras")

In [1]:
photo_path = "Varicela (52).jpg_0.png"

img_show = cv2.imread(photo_path)

cv2.imshow("Lesion Preview",img_show)
cv2.waitKey(0)
cv2.destroyAllWindows()

img = image.load_img(photo_path, target_size=(img_size, img_size))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
class_index = np.argmax(prediction)
confidence = np.max(prediction)

print(f"Predicted class: {class_index}, Confidence: {confidence:.2f}")


KeyboardInterrupt



In [59]:
val_generator.reset()
pred_probs = model.predict(val_generator, steps=len(val_generator))
pred_classes = np.argmax(pred_probs, axis=1)
true_classes = val_generator.classes

10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step


In [ ]:
f1 = f1_score(true_classes, pred_classes, average='weighted')
print(f"Weighted F1 Score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(true_classes, pred_classes, target_names=list(val_generator.class_indices.keys())))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, pred_classes))

10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 335ms/step
Weighted F1 Score: 0.3473

Classification Report:
              precision    recall  f1-score   support

      Herpes       0.17      0.37      0.23        46
    Melanoma       0.08      0.07      0.08        27
   MonkeyPox       0.31      0.35      0.33        83
    Varicela       0.57      0.35      0.44       162

    accuracy                           0.33       318
   macro avg       0.28      0.29      0.27       318
weighted avg       0.40      0.33      0.35       318


Confusion Matrix:
[[17  5 13 11]
 [10  2  7  8]
 [25  5 29 24]
 [47 12 46 57]]
